ICP algo :  

ICP directly operationalizes the assumption:

𝑃(𝑌∣𝑆,𝑒)= 𝑃(𝑌∣𝑆)∀𝑒

“Which subset of variables predicts ETA stably across cities/days?”

In [1]:
pip install causal-learn ecoml


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for GPUtil: filename=gputil-1.4.0-py3-none-any.whl size=7436 sha256=e135b6a65fd5317cac419d77c19c94f320a95d280f1ac5ccbffaac3185da7b12
  Stored in directory: /home/aymuos/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


- Z_raw = {
  road_distance,
  circuity,
  local_turn_density,
  rain_mm,
  snow_present,
  weather_regime,
  area_speed,
  region_speed_variance,
  stop_index_norm,
  active_orders,
  shift_duration,
  region_dwell
}
- Y = ETA
- e = (city, date)

# invariant causal prediction

In [ ]:
X = orders[Z_raw]
y = orders["ETA"]
e = orders["e"]   # environment labels

#TODO:Standardize features of X

stable_sets = []

for S in candidate_subsets:
    invariant = True
    for e1, e2 in environment_pairs:
        fit_e1 = regress(y[e==e1], X[S][e==e1])
        fit_e2 = regress(y[e==e2], X[S][e==e2])

        if not invariant_test(fit_e1, fit_e2):
            invariant = False
            break

    if invariant:
        stable_sets.append(S)

PA_Y = intersection_of(stable_sets)



# PC now 

-ICT gives 5-7 variables . now pc algorithm for verification 

In [ ]:
from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.cit import fisherz

data = np.column_stack([X_reduced, y])
cg = pc(data, alpha=0.05, indep_test=fisherz)

Step 1: Engineer causal features
Step 2: Define environments e = (city, date)
Step 3: Run ICP → get PA(Y)
Step 4: Define Z = PA(Y)
Step 5: (Optional) Run PC on Z ∪ {Y}
Step 6: Feed Z into contextual bandit